In [22]:
import pandas as pd

path_to_train = './data/train.csv'
path_to_test = './data/test2.csv'

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type'])
test = pd.read_csv(path_to_test, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type'])

df = pd.concat([train, test], ignore_index=True)
df.tail()

,ts_listen,listen_type,user_id
7578747,1479664431,1,12641
7578748,1480358099,1,10055
7578749,1480616853,1,1029
7578750,1479644510,1,4630
7578751,1480491885,1,6467


In [23]:
df = df.sort_values(by=['user_id', 'ts_listen'], axis=0)
df.tail()

,ts_listen,listen_type,user_id
7575803,1478016326,1,19915
569368,1477512816,0,19916
7563826,1478017434,1,19916
4791398,1477926978,0,19917
7565747,1478009685,1,19917


In [3]:
counter = 0
prev_user = 0
prev_listen_type = 0

def inline_iterate(row):
    #row = tp[1]
    global counter, prev_user, prev_listen_type
    
    if(row['listen_type'] != prev_listen_type):
        prev_listen_type = row['listen_type']
        counter = 0
        
    if(row['user_id'] != prev_user):
        prev_user = row['user_id']
        counter = 0
    
    if(counter < 10):
        counter += 1
        
    return counter

inline = [inline_iterate(row) for index,row in df.iterrows()]
#inline = list(map(inline_iterate, df.iterrows()))
inline

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 1

In [24]:
df['inline'] = inline

In [25]:
df['inline_flow'] = df.apply(lambda row: row['inline'] if row['listen_type'] == 1 else 0, axis=1)
df['inline_not_flow'] = df.apply(lambda row: row['inline'] if row['listen_type'] == 0 else 0, axis=1)

In [26]:
df.tail()

,ts_listen,listen_type,user_id,inline,inline_flow,inline_not_flow
7575803,1478016326,1,19915,2,2,0
569368,1477512816,0,19916,1,0,1
7563826,1478017434,1,19916,1,1,0
4791398,1477926978,0,19917,1,0,1
7565747,1478009685,1,19917,1,1,0


In [27]:
df = df.sort_index()
df.tail()

,ts_listen,listen_type,user_id,inline,inline_flow,inline_not_flow
7578747,1479664431,1,12641,1,1,0
7578748,1480358099,1,10055,4,4,0
7578749,1480616853,1,1029,6,6,0
7578750,1479644510,1,4630,1,1,0
7578751,1480491885,1,6467,10,10,0


In [30]:
test_inline = df.iloc[range(len(train), len(train)+len(test))][['inline', 'inline_flow', 'inline_not_flow']]
test_inline.to_csv('./data/features/inline.test.csv', encoding='utf-8', index=False)

train_inline = df.iloc[range(len(train))][['inline', 'inline_flow', 'inline_not_flow']]
train_inline.to_csv('./data/features/inline.train.csv', encoding='utf-8', index=False)

Спробуємо, чи є покращення?

In [31]:
from sklearn.ensemble import RandomForestClassifier
import validation as v
import model as m

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['genre_id', 'user_id', 'context_type', 'ts_listen', 'release_date', 'platform_name', 'platform_family', 'media_duration', 'listen_type', 'user_gender', 'is_listened'])
train = pd.concat([train, train_inline], axis=1)

train.head()

,genre_id,ts_listen,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,is_listened,inline,inline_flow,inline_not_flow
0,25471,1480597215,12,20040704,1,0,223,0,0,9241,0,10,0,10
1,25571,1480544735,0,20060301,2,1,171,0,0,16547,1,5,0,5
2,16,1479563953,1,20140714,2,1,149,1,1,7665,1,1,1,0
3,7,1480152098,0,20001030,0,0,240,0,1,1580,0,10,0,10
4,7,1478368974,0,20080215,0,0,150,0,1,1812,1,10,0,10
5,7,1478382544,0,20080215,0,0,150,0,1,1812,1,10,0,10
6,7,1478338409,0,20080215,0,0,150,0,1,1812,1,10,0,10
7,7,1478353709,1,20080215,1,0,150,1,1,10325,1,10,10,0
8,7,1479130924,0,20080215,0,0,150,0,1,1812,1,10,0,10
9,7,1479214304,1,20080215,1,2,150,1,1,51,1,10,10,0


In [32]:
x_cols = ['genre_id', 'user_id', 'context_type', 'ts_listen', 'release_date', 'platform_name', 'platform_family', 'media_duration', 'listen_type', 'user_gender', 'inline', 'inline_flow', 'inline_not_flow']
y_col = 'is_listened'
clf = RandomForestClassifier(n_estimators=30, random_state=42)

clf, auc = m.train_and_validate(train, x_cols, y_col, clf, v.only_flow_split_train_validation_by_last_record)
auc

0.61239106622979445